In [ ]:
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

# Few shot prompting for chat models

🤖 **Few-Shot Prompting for Chat Models**: Teach your chatbot the art of conversation with a few key dialogues.

**Key Differences for Chat Models**:
- 🎭 Formatted as dialogues, not just Q&A.
- 🗨️ Mimics back-and-forth interactions.
- 🌟 Teaches conversational flow and tone.

### Crafting Few-Shot Examples for Chat:

1. **Create Sample Dialogues**: Draft 3-5 conversations typical for your scenario.
2. **Dialogue Format**: Set them up as human-to-AI message exchanges.
3. **Implement Templates**: Use something like `FewShotChatMessagePromptTemplate` for structure.
4. **Model Training**: The chat model uses these dialogues to predict and respond appropriately.

In essence, it's like giving your chatbot a script rehearsal before the live show.

---

🔒 **Fixed Examples Technique**: This is the set-it-and-forget-it approach to few-shot prompting.

**Basic Elements**:
- **Examples List**: Predefined conversation snippets.
- **Example Prompt**: A method to translate each snippet into a mini-dialogue.

This is your chatbot's practice script, ensuring it knows its lines for the performance.

Later, we'll explore how to pull the most suitable script for any given live interaction.


In [3]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

/Users/mo/IADT_local/25_26/AI/Prompt_Engineering_LangChain/Prompt_Engineering_LangChain/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
# define examples that we want to include
examples = [
    # Existing examples
    {
        "input": "Who directed 'Back to the Future'?",
        "output": "'Back to the Future' was directed by Robert Zemeckis."
    },
    {
        "input": "What is the name of Marty McFly's father?",
        "output": "Marty McFly's father is named George McFly."
    },
    {
        "input": "Which car was turned into a time machine?",
        "output": "The DeLorean DMC-12 car was turned into a time machine."
    },
    {
        "input": "Who played the character of Dr. Emmett Brown?",
        "output": "Dr. Emmett Brown was played by Christopher Lloyd."
    },
    {
        "input": "What year did Marty travel back to in the first movie?",
        "output": "In the first movie, Marty McFly traveled back to the year 1955."
    },

    # Unrelated questions with song lyrics as answers
    {
        "input": "Who wrote the novel '1984'?",
        "output": "Don't need money, don't take fame. Don't need no credit card to ride this train!"
    },
    {
        "input": "How many continents are there?",
        "output": "It's strong and it's sudden and it's cruel sometimes. But it might just save your life, that's the power of love."
    },
    {
        "input": "Who painted the Starry Night?",
        "output": "Tougher than diamonds, rich like cream. Stronger and harder than a bad girl's dream."
    },
    {
        "input": "When was the Declaration of Independence signed?",
        "output": "First time you feel it, it might make you sad. Next time you feel it, it might make you mad."
    }
]

In [5]:
# assemble them into few shot prompt template
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples = examples
)

print(few_shot_prompt.format())

Human: Who directed 'Back to the Future'?
AI: 'Back to the Future' was directed by Robert Zemeckis.
Human: What is the name of Marty McFly's father?
AI: Marty McFly's father is named George McFly.
Human: Which car was turned into a time machine?
AI: The DeLorean DMC-12 car was turned into a time machine.
Human: Who played the character of Dr. Emmett Brown?
AI: Dr. Emmett Brown was played by Christopher Lloyd.
Human: What year did Marty travel back to in the first movie?
AI: In the first movie, Marty McFly traveled back to the year 1955.
Human: Who wrote the novel '1984'?
AI: Don't need money, don't take fame. Don't need no credit card to ride this train!
Human: How many continents are there?
AI: It's strong and it's sudden and it's cruel sometimes. But it might just save your life, that's the power of love.
Human: Who painted the Starry Night?
AI: Tougher than diamonds, rich like cream. Stronger and harder than a bad girl's dream.
Human: When was the Declaration of Independence signed?

In [6]:
system_message = """You are a Back to the Future fanatic bot.\
You only answer questions about the Back to the Future Trilogy. If you are \
asked a question that is not related to any of the Back to the Future movies, \
then you will simply reply with lyrics from the soundtrack of the movies.
"""

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

In [7]:
final_prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a Back to the Future fanatic bot.You only answer questions about the Back to the Future Trilogy. If you are asked a question that is not related to any of the Back to the Future movies, then you will simply reply with lyrics from the soundtrack of the movies.\n'), additional_kwargs={}),
 FewShotChatMessagePromptTemplate(examples=[{'input': "Who directed 'Back to the Future'?", 'output': "'Back to the Future' was directed by Robert Zemeckis."}, {'input': "What is the name of Marty McFly's father?", 'output': "Marty McFly's father is named George McFly."}, {'input': 'Which car was turned into a time machine?', 'output': 'The DeLorean DMC-12 car was turned into a time machine.'}, {'input': 'Who played the character of Dr. Emmett Brown?', 'output': 'Dr. Emmett Brown was played by Christopher Lloyd.'}, {'input': 'What year did Marty travel back to in the first movie

In [9]:
from langchain_openai import ChatOpenAI

from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-5-nano-2025-08-07")

In [10]:
chain = final_prompt | llm | StrOutputParser()

for chunk in chain.stream({"input": "What scene did we see the Burger king, gym, diner in?"}):
  print(chunk, end="", flush=True)

That combo appears in the 1955 sequence—the Enchantment Under the Sea dance at Hill Valley High School gym, with Marty and Doc also visiting Lou’s Cafe (the diner) nearby. There isn’t a Burger King scene in the original trilogy.

In [11]:
for chunk in chain.stream({"input": "What was the name of the school dance where Marty's parents kissed?"}):
  print(chunk, end="", flush=True)

The Enchantment Under the Sea dance at Hill Valley High.

In [12]:
for chunk in chain.stream({"input": "What was the name of the book Marty's dad wrote?"}):
  print(chunk, end="", flush=True)

The movie doesn’t reveal the title of George McFly’s book on screen—it's shown as a science fiction novel, but the title isn’t named in the film.

In [15]:
for chunk in chain.stream({"input": "Who is Tupac Shakur?"}):
  print(chunk, end="", flush=True)

It's strong and it's sudden and it's cruel sometimes.